# Tutorial まずはRAGをやってみる
1,LangChainを使ったRAGのチュートリアル

## Load Document
ドキュメントを読み込みます。今回はChatgptで作成した社内文書を模した簡易的なデータセット(company_documents_dataset_1)からフォルダ単位で読み込み

In [13]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../datasets/company_documents_dataset_1/", glob="**/*.txt",recursive=True)
raw_docs = loader.load()
print("ドキュメントの数:",len(raw_docs))
print(raw_docs[0])

ドキュメントの数: 5
<class 'langchain_core.documents.base.Document'>
page_content='# 社宅・寮管理規程\n\n## 第１条（目的）\n\nこの規程は、社員の居住のために会社が所有する社宅または寮および会社名義で借り上げた社宅または寮の管理運営に関する事項を定めたものである。\n\n## 第２条（入居資格）\n\n社宅への入居資格は、配偶者または同居する家族がいる社員とする。\n\n寮への入居資格は、独身者であって自宅から通勤困難な社員とする。\n\n## 第３条（入居への申し込み）\n\n1. 社宅または寮への入居を希望する社員は「社宅・寮入居申込書」に必要事項を記入の上、所属長を経由して漆黒 花太郎へ申し込むものとする。\n\n2. 社宅または寮への入居を許可された社員は、直ちに「社宅・寮入居誓約書」を会社に提出しなければならない。\n\n## 第４条（借上社宅・寮）\n\n社宅または寮への入居希望があるにもかかわらず、入居可能な社宅または寮がない場合は、民間の賃貸住宅を社宅または寮として借り上げることがある。\n\n## 第５条（借上社宅・寮の手配）\n\n借上社宅または寮の手配は会社が行い、家主との間に契約を締結する。\n\n## 第６条（借上社宅の間取り）\n\n借上げ社宅または寮として利用する住宅の間取りは次の区分による。\n\n子がいる場合：３ＤＫ\n\n配偶者のみの場合：２ＤＫ\n\n独身者：１Ｋ\n\n前項の規程に関わらず、扶養家族等の人数により間取りを変更することがある。\n\n## 第７条（借上社宅・寮の家賃限度額）\n\n借上げ社宅または寮の家賃限度額は、次の通りとする。\n\n子がいる場合：100000円\n\n配偶者のみの場合：50000円\n\n独身者：0円\n\n前項の家賃限度額を超過した場合、その超過分については入居者が負担する。\n\n## 第８条（社宅の使用料）\n\n会社所有の社宅または寮の使用料は次の通りとする。\n\n社宅：120000円\n\n寮：80000円\n\n中途入居、中途退去の場合で１か月に満たないときは、日割計算による。\n\n## 第９条（入居資格の喪失）\n\n社宅入居者が次の各号のいずれかに該当した場合は、入居資格を喪失し、会社が定める期

##  Text Split 
MarkdownHeaderTextSplitter + RecursiveCharacterTextSplitterの二つを使ってテキストを分割。

In [7]:
from langchain.text_splitter import MarkdownHeaderTextSplitter,RecursiveCharacterTextSplitter
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, 
    return_each_line=False,
    strip_headers = False 
)
docs = []
for raw_doc in raw_docs:
    source = raw_doc.metadata["source"]
    spilited_docs = markdown_splitter.split_text(raw_doc.page_content)
    for doc in spilited_docs:
        doc.metadata["source"] = source#metadataにフォルダのパスを加える
    docs = docs + spilited_docs

markdown_splited_docs = docs
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300,chunk_overlap=50)
docs = text_splitter.split_documents(docs)
for doc in docs:
    print("【page_content】")
    print(doc.page_content)
    print("【metadata】")
    print(doc.metadata)
    print("-"*180)
print("markdown分割後のドキュメントの数:",len(markdown_splited_docs))
print("RecursiveCharacterTextSplitterで分割後のドキュメントの数:",len(docs))

【page_content】
# 社宅・寮管理規程  
## 第１条（目的）  
この規程は、社員の居住のために会社が所有する社宅または寮および会社名義で借り上げた社宅または寮の管理運営に関する事項を定めたものである。
【metadata】
{'Header 1': '社宅・寮管理規程', 'Header 2': '第１条（目的）', 'source': '../datasets/company_documents_dataset_1/規定及び規則/社宅管理規定.txt'}
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
【page_content】
## 第２条（入居資格）  
社宅への入居資格は、配偶者または同居する家族がいる社員とする。  
寮への入居資格は、独身者であって自宅から通勤困難な社員とする。
【metadata】
{'Header 1': '社宅・寮管理規程', 'Header 2': '第２条（入居資格）', 'source': '../datasets/company_documents_dataset_1/規定及び規則/社宅管理規定.txt'}
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
【page_content】
## 第３条（入居への申し込み）  
1. 社宅または寮への入居を希望する社員は「社宅・寮入居申込書」に必要事項を記入の上、所属長を経由して漆黒 花太郎へ申し込むものとする。  
2. 社宅または寮への入居を許可された社員は、直ちに「社宅・寮入居誓約書」を会社に提出しなければならない。
【metadata

## Embedding

In [8]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
vectorstore = Chroma.from_documents(persist_directory="./vecstore/index", documents=docs, embedding=OpenAIEmbeddings())

/Users/rokuroyamaki/anaconda3/envs/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## similarity_search
ちゃんとベクトル検索ができるか確認

In [9]:
question = "社長の名前"
vectorstore = Chroma(persist_directory="./vecstore/index", embedding_function=OpenAIEmbeddings()) #vectorstoreを読み込み
result_docs = vectorstore.similarity_search(question,k=3)
for doc  in  result_docs:
    print(doc)

page_content='### 社長  \n漆黒 花太郎（しっこく かたろう）' metadata={'Header 1': '株式会社架空ブラック 会社情報', 'Header 2': '会社概要', 'Header 3': '社長', 'source': '../datasets/company_documents_dataset_1/マニュアル/会社情報.txt'}
page_content='## 社長のプロフィール  \n### 名前  \n漆黒 花太郎（しっこく かたろう）' metadata={'Header 1': '株式会社架空ブラック 会社情報', 'Header 2': '社長のプロフィール', 'Header 3': '名前', 'source': '../datasets/company_documents_dataset_1/マニュアル/会社情報.txt'}
page_content='# 第４条（役職手当）  \n## １. 役職手当は、次の金額とします。  \n部長：50,000円  \n課長：10,000円' metadata={'Header 1': '第４条（役職手当）', 'Header 2': '１. 役職手当は、次の金額とします。', 'source': '../datasets/company_documents_dataset_1/規定及び規則/給与規則.txt'}


## Search and Generate
ドキュメントを検索して、そのドキュメントをもとにChatGPT3.５で回答します

In [12]:
from langchain.retrievers import RePhraseQueryRetriever
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

question = "社長の名前は？"

prompt = PromptTemplate(
    input_variables=["context","question"],
    template="""以下の参考用のテキストの一部を参照して、Questionに回答してください。もし参考用のテキストの中に回答に役立つ情報が含まれていなければ、分からない、と答えてください。
{context}
Question: {question}
Answer: """
)
llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
vectorstore = Chroma(persist_directory="./vecstore/index", embedding_function=OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
qa_chain = RetrievalQA.from_chain_type(llm,retriever=retriever,
                                           chain_type="stuff", 
                                           return_source_documents=True,
                                           verbose=True,
                                           chain_type_kwargs={"prompt":prompt})
with get_openai_callback() as cb:
    result = qa_chain({"query": question})
    print("【質問】:",result["query"],"  【回答】:",result["result"])
    print("")
    print("【回答に使用したドキュメント】")
    if result.get('source_documents'):
        for source  in result['source_documents']:
            print(source)
            print("")
    print("【トークン数】")
    print(cb)



> Entering new RetrievalQA chain...

> Finished chain.
【質問】: 社長の名前は？   【回答】: 漆黒 花太郎（しっこく かたろう）

【回答に使用したドキュメント】
page_content='### 社長  \n漆黒 花太郎（しっこく かたろう）' metadata={'Header 1': '株式会社架空ブラック 会社情報', 'Header 2': '会社概要', 'Header 3': '社長', 'source': '../datasets/company_documents_dataset_1/マニュアル/会社情報.txt'}

page_content='## 社長のプロフィール  \n### 名前  \n漆黒 花太郎（しっこく かたろう）' metadata={'Header 1': '株式会社架空ブラック 会社情報', 'Header 2': '社長のプロフィール', 'Header 3': '名前', 'source': '../datasets/company_documents_dataset_1/マニュアル/会社情報.txt'}

page_content='# 第４条（役職手当）  \n## １. 役職手当は、次の金額とします。  \n部長：50,000円  \n課長：10,000円' metadata={'Header 1': '第４条（役職手当）', 'Header 2': '１. 役職手当は、次の金額とします。', 'source': '../datasets/company_documents_dataset_1/規定及び規則/給与規則.txt'}

page_content='### 従業員数  \n約50名' metadata={'Header 1': '株式会社架空ブラック 会社情報', 'Header 2': '会社概要', 'Header 3': '従業員数', 'source': '../datasets/company_documents_dataset_1/マニュアル/会社情報.txt'}

【トークン数】
Tokens Used: 247
	Prompt Tokens: 225
	Completion Tokens: 22
Successful Re